# 20220114
Trying to follow Yam Peleg's Time Series Transformer + Time2Vec implementation (which is in Keras -- see [here](https://www.kaggle.com/yamqwe/tutorial-time-series-transformer-time2vec)) 


In [1]:
# notebook configuration
# if '/sf/' in pwd:
#     COLAB, SAGE = False, False
# elif 'google.colab' in str(get_ipython()):
#     COLAB, SAGE = True, False # do colab-specific installs later
# else:
#     COLAB, SAGE = False, True
    
CONTEXT = 'local' # or 'colab', 'sage', 'kaggle'
USE_GPU = True 
%config Completer.use_jedi = False

## Imports

In [2]:
# basic imports
from pathlib import Path
import os
import math
from datetime import datetime
import random

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

import requests # for telegram notifications
from tqdm.notebook import tqdm

from joblib import dump, load

Now, non-stdlib imports

In [3]:
# model selection
from sklearn.model_selection import train_test_split, StratifiedKFold, KFold

# metrics
# from sklearn.metrics import accuracy_score#, log_loss, roc_auc_score

# eda
import missingno
# import doubtlab 

# data cleaning
# from sklearn.impute import SimpleImputer #, KNNImputer
# import cleanlab

# normalization
# from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, MaxAbsScaler, RobustScaler, QuantileTransformer
# from gauss_rank_scaler import GaussRankScaler

# feature generation
# from sklearn.preprocessing import PolynomialFeatures
# import category_encoders as ce

# models
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import StackingClassifier, RandomForestClassifier

# feature reduction
# from sklearn.decomposition import PCA
# from umap import UMAP

# clustering
# from sklearn.cluster import DBSCAN, KMeans
# import hdbscan

# feature selection
# from sklearn.feature_selection import SelectKBest, f_regression, mutual_info_regression
# import featuretools as ft
# from BorutaShap import BorutaShap
# from boruta import BorutaPy

# tracking 
import wandb
from wandb.xgboost import wandb_callback
from wandb.lightgbm import wandb_callback
os.environ['WANDB_NOTEBOOK_NAME'] = f"nb_{datetime.now().strftime('%Y%m%d')}.ipynb"

In [4]:
# deep learning
import torch
from torch.optim import Adam, AdamW, Adagrad, SGD, RMSprop, LBFGS
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts, CyclicLR, OneCycleLR, StepLR, CosineAnnealingLR
import torch.nn as nn
# widedeep
# from pytorch_widedeep import Trainer
# from pytorch_widedeep.preprocessing import WidePreprocessor, TabPreprocessor
# from pytorch_widedeep.models import Wide, TabMlp, WideDeep, SAINT#, TabTransformer, TabNet, TabFastFormer, TabResnet
# from pytorch_widedeep.metrics import Accuracy
# from pytorch_widedeep.callbacks import EarlyStopping, LRHistory, ModelCheckpoint

In [5]:
# time series
# import tsfresh

# import darts
# from darts import TimeSeries

In [6]:
# from darts.models import ExponentialSmoothing, AutoARIMA, ARIMA, Prophet, RandomForest, RegressionEnsembleModel, RegressionModel, TFTModel, TCNModel, TransformerModel, NBEATSModel

## Routing

Now, datapath setup

In [7]:
if CONTEXT == 'colab':
    # mount Google Drive
    from google.colab import drive
    drive.mount('/content/drive')
    
    # handling datapath
    # datapath = Path('/content/drive/MyDrive/kaggle/tabular_playgrounds/dec2021/')
    root = Path('') # TODO

elif CONTEXT == 'sage':
    root = Path('') # TODO
    
elif CONTEXT == 'kaggle':
    root = Path('') # TODO
    
else: # if on local machine
    root = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/jan2022/')
    datapath = root/'datasets'
    # edapath = root/'EDA'
    # modelpath = Path('/media/sf/easystore/kaggle_data/tabular_playgrounds/oct2021/models/')
    predpath = root/'preds'
    subpath = root/'submissions'
    studypath = root/'studies'
    
    for pth in [datapath, predpath, subpath, studypath]:
        pth.mkdir(exist_ok=True)

## Helpers

In [8]:
SEED = 42

# Function to seed everything but the models
def seed_everything(seed, pytorch=True, reproducible=True):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    if pytorch:
        torch.manual_seed(seed) # set torch CPU seed
        if torch.cuda.is_available():
            torch.cuda.manual_seed_all(seed) # set torch GPU(s) seed(s)
        if reproducible and torch.backends.cudnn.is_available():
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False

seed_everything(seed=SEED)

In [9]:
def reduce_memory_usage(df, verbose=True):
    """
    Function to reduce memory usage by downcasting datatypes in a Pandas DataFrame when possible.
    
    h/t to Bryan Arnold (https://www.kaggle.com/puremath86/label-correction-experiments-tps-nov-21)
    """
    
    numerics = ["int8", "int16", "int32", "int64", "float16", "float32", "float64"]
    start_mem = df.memory_usage().sum() / 1024 ** 2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == "int":
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if (
                    c_min > np.finfo(np.float16).min
                    and c_max < np.finfo(np.float16).max
                ):
                    df[col] = df[col].astype(np.float16)
                elif (
                    c_min > np.finfo(np.float32).min
                    and c_max < np.finfo(np.float32).max
                ):
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024 ** 2
    if verbose:
        print(
            "Mem. usage decreased to {:.2f} Mb ({:.1f}% reduction)".format(
                end_mem, 100 * (start_mem - end_mem) / start_mem
            )
        )
    return df

In [10]:
tg_api_token = 'your_api_token' # for Galileo (jupyter_watcher_bot) on Telegram
tg_chat_id = 'your_chat_id'

import requests

def send_tg_message(text='Cell execution completed.'):  
    """
    h/t Ivan Dembicki Jr. for the base version 
    (https://medium.com/@ivan.dembicki.jr/notifications-in-jupyter-notebook-with-telegram-f2e892c55173)
    """
    requests.post('https://api.telegram.org/' +  'bot{}/sendMessage'.format(tg_api_token),
                  params=dict(chat_id=tg_chat_id, text=text))

In [11]:
def SMAPE(y_true, y_pred):
    '''
    h/t Jean-François Puget (@CPMP) -- see https://www.kaggle.com/c/web-traffic-time-series-forecasting/discussion/36414
    '''
    denominator = (y_true + np.abs(y_pred)) / 200.0
    diff = np.abs(y_true - y_pred) / denominator
    diff[denominator == 0] = 0.0
    return np.mean(diff)

## Dataset Setup

In [12]:
# dataset_params will initially include either trivial class instances or loaded, precomputed artifacts
dataset_params = {
    'train_source': str(datapath/'train.csv'),
    'target_source': str(datapath/'train.csv'),
    'test_source': str(datapath/'test.csv'),
    # 'scaler': str(RobustScaler()),
    # 'pca': str(load(datapath/'pca_mle-RobustScaled_orig_trainset.joblib')),
    # 'umap': str(load(datapath/'umap_reducer-20211107-n_comp10-n_neighbors15-rs42-pca_mle-RobustScaled_orig_trainset.joblib')),
}   

# referring back to the already-entered attributes, specify how the pipeline was sequenced
# dataset_params['preprocessing_pipeline'] = str([dataset_params['scaler'], dataset_params['pca'], dataset_params['umap']]) # ACTUALLY this is unwieldy
# dataset_params['preprocessing_pipeline'] = '[scaler, pca, umap]' # more fragile, but also more readable

# now, load the datasets and generate more metadata from them
train_df = pd.read_csv(datapath/'train.csv')
test_df = pd.read_csv(datapath/'test.csv')

Following Yam Peleg, I'll combine all the data together for ease of transforms.

In [13]:
all_data = pd.concat([train_df, test_df])

Now, I'll manually create time features -- no holidays (at least for now), however. Perhaps later...

In [14]:
all_data['date'] = pd.to_datetime(all_data['date'])
all_data['year'] = all_data['date'].dt.year
all_data['month'] = all_data['date'].dt.month
all_data['day'] = all_data['date'].dt.day
all_data['dayofweek'] = all_data['date'].dt.dayofweek
all_data['dayofmonth'] = all_data['date'].dt.days_in_month
all_data['dayofyear'] = all_data['date'].dt.dayofyear
all_data['weekday'] = all_data['date'].dt.weekday
all_data['weekofyear'] = all_data['date'].dt.weekofyear
all_data.drop(columns = ['num_sold', 'date', 'row_id'], inplace = True)
all_data = pd.get_dummies(all_data)

In [15]:
all_data.columns

Index(['year', 'month', 'day', 'dayofweek', 'dayofmonth', 'dayofyear',
       'weekday', 'weekofyear', 'country_Finland', 'country_Norway',
       'country_Sweden', 'store_KaggleMart', 'store_KaggleRama',
       'product_Kaggle Hat', 'product_Kaggle Mug', 'product_Kaggle Sticker'],
      dtype='object')

- Note that the call to `pd.get_dummies` eliminates the object/string categorical columns, replacing them with one-hots -- it doesn't simply add the one-hots.

In [16]:
y = train_df['num_sold'].values
y_orig = train_df['num_sold'].values

Now, we break the dataset back apart:

In [17]:
train_df = all_data[:len(train_df)]
test_df = all_data[len(train_df):]

In [18]:
len(train_df), len(test_df)

(26298, 6570)

Now, we define the rolling window helper function, which will help construct a vector suitable for the DNN.

In [19]:
def rolling_window(df, y = None, window_size = 10):
    all_features, all_targets = [], []
    for i in range(0, len(df) - window_size):
        all_features.append(np.expand_dims(df[i: i + window_size].values, axis = 0))
        if y is not None: all_targets.append(np.expand_dims(y[i + window_size], axis = 0))
    if y is not None: return np.concatenate(all_features, axis = 0), np.concatenate(all_targets, axis = 0)
    else: return np.concatenate(all_features, axis = 0)

# Model Architecture

Now, let's define the Transformer Block. Some notes...
- The `nn.MultiheadAttention` layer expects an `embed_dim=` argument; the `kdim=` argument defaults to `None`, which is to say it sets `kdim=` to the value `embed_dim`. So, I think my code below is equivalent to the Keras `self.att = layers.MultiHeadAttention(num_heads = num_heads, key_dim = embed_dim)`.
- `nn.Linear` is equivalent to Keras's `layers.Dense`, with two difference: 1) you have to explicitly give an input dimension to PyTorch, whereas in Keras you only have to supply an output dimension, and 2) you separate out the activation function in PyTorch. I think, then that my `nn.Linear(in_features=embed_dim, out_features=ff_dim)` followed by `nn.GeLU` is equivalent to Yam Peleg's `layers.Dense(ff_dim, activation = "gelu")`.
- Not sure of the `nn.BatchNorm2d`...

In [20]:
nn.BatchNorm1d?

Init signature:
nn.BatchNorm1d(
    num_features,
    eps=1e-05,
    momentum=0.1,
    affine=True,
    track_running_stats=True,
)
Docstring:     
Applies Batch Normalization over a 2D or 3D input (a mini-batch of 1D
inputs with optional additional channel dimension) as described in the paper
`Batch Normalization: Accelerating Deep Network Training by Reducing
Internal Covariate Shift <https://arxiv.org/abs/1502.03167>`__ .

.. math::

    y = \frac{x - \mathrm{E}[x]}{\sqrt{\mathrm{Var}[x] + \epsilon}} * \gamma + \beta

The mean and standard-deviation are calculated per-dimension over
the mini-batches and :math:`\gamma` and :math:`\beta` are learnable parameter vectors
of size `C` (where `C` is the input size). By default, the elements of :math:`\gamma` are set
to 1 and the elements of :math:`\beta` are set to 0. The standard-deviation is calculated
via the biased estimator, equivalent to `torch.var(input, unbiased=False)`.

Also by default, during training this layer keeps running es

In [21]:
class TransformerBlock(nn.Module):
    def __init__(self, embed_dim, feat_dim, num_heads=8, ff_dim=256, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads)
        self.ffn = nn.Sequential(
            nn.Linear(in_features=embed_dim, out_features=ff_dim),
            nn.GELU(),
            nn.Linear(in_features=ff_dim, out_features=feat_dim)
        )
        self.layernorm1 = nn.BatchNorm1d() # or 2d?
        self.layernorm2 = nn.BatchNorm1d() # or 2d?
        self.dropout1 = nn.Dropout()
        self.dropout2 = nn.Dropout()
        
    def forward(self, inputs, training):
        attn_output = self.att(inputs)
        # TODO:finish

I'm adapting `Time2Vec` from [this @ojus1 repo](https://github.com/ojus1/Time2Vec-PyTorch) rather than trying to clone Yam Peleg's. He implements a function that expresses the linear combination of $\omega_i \tau + \varphi_i$ for when $i=0$ and then wraps it, if appropriate, in a periodic activation function $\mathcal{F}$ -- which can be either sine or cosine. Recall that the full equation is $$t2v(\tau)[i] = \left\{\begin{array}{ll} \omega_i\tau + \varphi_i & \text{if }i=0 \\ \mathcal{F}(\omega_i \tau + \varphi_i) & \text{if } 1 \leq i \leq k\end{array}\right.$$where $k$ is the time2vec dimension (that is, the dimensionality of the embedding in latent space), $\tau$ is a raw [[time series]], $\mathcal{F}$ is a periodic [[activation function]], and $\omega$ and $\varphi$ are learnable params.

In [22]:
import math

In [23]:
math.sin(0)

0.0

In [24]:
math.cos(0)

1.0

In [21]:
def t2v(tau, f, out_features, w, b, w0, b0, arg=None):
    # https://github.com/ojus1/Time2Vec-PyTorch/blob/master/periodic_activations.py
    if arg:
        v1 = f(torch.matmul(tau, w) + b, arg)
    else:
        #print(w.shape, t1.shape, b.shape)
        v1 = f(torch.matmul(tau, w) + b)
    v2 = torch.matmul(tau, w0) + b0
    #print(v1.shape)
    return torch.cat([v1, v2], 1)

In [22]:
class SineActivation(nn.Module):
    # https://github.com/ojus1/Time2Vec-PyTorch/blob/master/periodic_activations.py
    def __init__(self, in_features, out_features):
        super(SineActivation, self).__init__()
        self.out_features = out_features
        self.w0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        self.b0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        self.w = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        self.b = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        self.f = torch.sin

    def forward(self, tau):
        return t2v(tau, self.f, self.out_features, self.w, self.b, self.w0, self.b0)

In [23]:
class CosineActivation(nn.Module):
    # https://github.com/ojus1/Time2Vec-PyTorch/blob/master/periodic_activations.py
    def __init__(self, in_features, out_features):
        super(CosineActivation, self).__init__()
        self.out_features = out_features
        self.w0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        self.b0 = nn.parameter.Parameter(torch.randn(in_features, 1))
        self.w = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        self.b = nn.parameter.Parameter(torch.randn(in_features, out_features-1))
        self.f = torch.cos

    def forward(self, tau):
        return t2v(tau, self.f, self.out_features, self.w, self.b, self.w0, self.b0)

In [ ]:
# if __name__ == "__main__":
#     sineact = SineActivation(1, 64)
#     cosact = CosineActivation(1, 64)

#     print(sineact(torch.Tensor([[7]])).shape)
#     print(cosact(torch.Tensor([[7]])).shape)

The wrapper then looks like so (from [here](https://github.com/ojus1/Time2Vec-PyTorch/blob/master/Model.py):

In [24]:
class Time2Vec(nn.Module):
    # https://github.com/ojus1/Time2Vec-PyTorch/blob/master/Model.py
    def __init__(self, activation, hidden_dim):
        super(Time2Vec, self).__init__()
        if activation == 'sin':
            self.l1 = SineActivation(1, hidden_dim)
        elif activation == 'cos':
            self.l1 = CosineActivation(1, hidden_dim)
        # note that if no activation function is supplied, the linear combo is used
        self.fc1 = nn.Linear(hidden_dim, 2) 
        
    def forward(self,x):
        # x = x.unsqueeze(1) # for testing without batch numbers
        x = self.l1(x)
        x = self.fc1(x)
        return x

In [15]:
# dataset_params['feature_count'] = X.shape[1]
# dataset_params['instance_count'] = X.shape[0]

# # might eventually shift from dict to tuple

# # simplest approach: k-v where key is new feature, v is string with the operation to get it
# # sacrifices sortability, but could recover that through regexes, and it's much quicker to input
# dataset_params['feature_combinations'] = {
#     'EHiElv': "df['Horizontal_Distance_To_Roadways'] * df['Elevation']",
#     'EViElv': "df['Vertical_Distance_To_Hydrology'] * df['Elevation']",
#     'EVDtH': "df.Elevation - df.Vertical_Distance_To_Hydrology",
#     'EHDtH': "df.Elevation - df.Horizontal_Distance_To_Hydrology * 0.2",
#     'Euclidean_Distance_to_Hydrology': "(df['Horizontal_Distance_To_Hydrology']**2 + df['Vertical_Distance_To_Hydrology']**2)**0.5",
#     'Manhattan_Distance_to_Hydrology': "df['Horizontal_Distance_To_Hydrology'] + df['Vertical_Distance_To_Hydrology']",
#     'Hydro_Fire_1': "df['Horizontal_Distance_To_Hydrology'] + df['Horizontal_Distance_To_Fire_Points']",
#     'Hydro_Fire_2': "abs(df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Fire_Points'])",
#     'Hydro_Road_1': "abs(df['Horizontal_Distance_To_Hydrology'] + df['Horizontal_Distance_To_Roadways'])",
#     'Hydro_Road_2': "abs(df['Horizontal_Distance_To_Hydrology'] - df['Horizontal_Distance_To_Roadways'])",
#     'Fire_Road_1': "abs(df['Horizontal_Distance_To_Fire_Points'] + df['Horizontal_Distance_To_Roadways'])",
#     'Fire_Road_2': "abs(df['Horizontal_Distance_To_Fire_Points'] - df['Horizontal_Distance_To_Roadways'])"
# }

# dataset_params['feature_clipping'] = [
#     {
#         'features': ['Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm'],
#         'range': range(0,256)
#     },
#     {
#         'features': ['Aspect'],
#         'range': range(0,360)
#     }
# ]

# # the features that are just getting the one-hots counted
# dataset_params['feature_counts'] = ['Soil_Type*', 'Wilderness_Area*']
# dataset_params['feature_complements'] = [
#     {
#         'old': 'Aspect', 
#         'new': 'Aspect2',
#         'operation': 'If x < 180 return x-180, else return x + 180'
#     },
# ]

# dataset_params['feature_indicators'] = {
#     'Hillshade_3pm_is_zero': "(df.Hillshade_3pm == 0).astype(int)",
# }

# dataset_params['feature_typecasting'] = {
#     'Highwater': "(df.Vertical_Distance_To_Hydrology < 0).astype(int)"
# }

# dataset_params['feature_encodings'] = "Soil_Type* features concatenated into single 40-bit integers and then five 8-bit integers, and finally to five decimals; see gbms_20211223.ipynb and the section 'Encoding the `Soil_Type` Features'."
# dataset_params['feature_removals'] = "Soil_Type* features removed after being encoded"


## Training Params

In [16]:
# training_params = {
#     'general_random_state': SEED,
# }

# folds = 5
# training_params['cross_val_strategy'] = StratifiedKFold(n_splits=folds, shuffle=True, random_state=SEED)

## Model Params

## Metadata

In [ ]:
# # baseline -- alter as needed later
# exmodel_config = {
#     'general_random_state': SEED,
# #     'feature_generation': ['NaN_counts', 'SummaryStats', 'NaN_OneHots'],
#     **dataset_params,
# #     **training_params,
# #     **model_params # perhaps do later
# }

## WandB Config

In [ ]:
# # wandb config:
# wandb_config = {
#     'name': f"{os.environ['WANDB_NOTEBOOK_NAME'][:-6]}_{datetime.now().strftime('%H%M%S')}", # just removes the .ipynb extension, leaving the notebook filename's stem
#     'tags': ['EDA'],
#     'notes': "EDA"
# }